In [ ]:
pip install beautifulsoup4 pyowm

In [11]:
import pandas as pd
from bs4 import BeautifulSoup

# Define file path
file_path = 'hotel-info-chapter8-001.csv'

# Define the path for the cleaned file
cleaned_file_path = 'hotel-info-chapter8-001_cleaned.csv'

# Load dataset
df = pd.read_csv(file_path, encoding='utf-8')


# Function to remove HTML tags
def remove_html_tags(text):
    if pd.isna(text):
        return ""  # Return an empty string for NaN values
    elif isinstance(text, str):
        return BeautifulSoup(text, "html.parser").get_text()
    else:
        return str(text)  # Convert non-string, non-NaN values to string

# Apply the function to clean HTML tags from the Description column
df['Description'] = df['Description'].apply(remove_html_tags)
df['Attractions'] = df['Attractions'].apply(remove_html_tags)

# Save the cleaned DataFrame to a new CSV file
df.to_csv(cleaned_file_path, index=False, encoding='utf-8')

C:\Users\AdrianThomspon\AppData\Local\Temp\ipykernel_23156\2320623278.py:19: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()


In [10]:
import langchain
from langchain_core.documents.base import Document
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load CSV
loader = CSVLoader(
    file_path="hotel-info-chapter8-001_cleaned.csv",source_column="HotelName",metadata_columns=["countryCode","countryName","cityCode","cityName","HotelCode","HotelName","HotelRating","Address","Attractions","Map","PhoneNumber","PinCode","HotelWebsiteUrl"])

docs = loader.load()

# Create splitter
splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=20)

# Define columns to index
content_columns = ["Description", "Reviews"]

# Split rows into chunks, focusing on 'Description' and 'Reviews' columns

chunked_docs = []

for doc in docs:
    chunked_doc_parts = []
    
    content_dict = {}
    # creating dictionary from the page_content so it's easier to access the column values for chunking
    pairs = doc.page_content.split('\n')
    for pair in pairs:
        if ': ' in pair:  # Check if the pair is valid
            key, value = pair.split(': ', 1)  # Split by the first colon only
            content_dict[key.strip()] = value.strip()
    for column in content_columns:
        
        # Check if the column exists and is not empty
        if column in content_dict and content_dict[column]:
            # Split the content of the column
            chunks = splitter.split_text(content_dict[column])
            chunked_doc_parts.extend(chunks)
            # Create the document as a dictionary
    chunked_docs.extend([Document(page_content=doc_part, metadata=doc.metadata) for doc_part in chunked_doc_parts])
    


for doc in chunked_docs:
    print(doc)
print("chunked_docs" + str(len(chunked_docs)))

page_content='Apartament acollidor, nestled in the picturesque town of Arinsal, offers guests a serene retreat with stunning mountain views. This charming apartment is just 6 km away from the vibrant Arinsal (Ski Station Pal-Arinsal), making it an ideal base for' metadata={'source': 'Apartament Acollidor', 'row': 0, 'countryCode': 'AD', 'countryName': 'Andorra', 'cityCode': '109855', 'cityName': 'Arinsal', 'HotelCode': '5801488', 'HotelName': 'Apartament Acollidor', 'HotelRating': 'All', 'Address': 'Crta Mas De Ribafeta 17', 'Attractions': '', 'Map': '42.56999|1.48912', 'PhoneNumber': '', 'PinCode': 'AD400', 'HotelWebsiteUrl': 'https://www.booking.com/hotel/ad/apartament-acollidor.html'}
page_content='an ideal base for skiing enthusiasts and nature lovers alike. With free WiFi and private parking on-site, guests can enjoy the convenience and connectivity during their stay.' metadata={'source': 'Apartament Acollidor', 'row': 0, 'countryCode': 'AD', 'countryName': 'Andorra', 'cityCode': 

In [11]:
len(chunked_docs)

1217

In [31]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
os.environ['OPENAI_API_KEY'] = ''
  
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small") 

chroma_document_store = Chroma( 
    collection_name="chapter8db_hotel_store", 
    embedding_function=embeddings_model, 
    persist_directory="streamlit/chapter8db",
)
similar_documents = chroma_document_store.similarity_search(query='hotels near Greenwich, London, UK',k=10)
similar_documents
#chroma_document_store.delete_collection()

#chroma_document_store.add_documents(chunked_docs)
#chroma_document_store.persist()

[Document(page_content='London Quiet room  |  Good location Greenwich and Blackheath are great plus Canary Wharf is a 10 min DLR ride away and accessible to the rest of London within 40 mins plus all usual amenities such as gym bar and restaurant Reception staff helpful', metadata={'Address': '1 Jalan Malinja Taman Malinja Taman MalinjaBukit Baru75150 Melaka', 'Attractions': 'Distances are displayed to the nearest 0.1 mile and kilometer.  Malacca General Hospital - 1.1 km / 0.7 mi  Putra Specialist Hospital - 2.2 km / 1.4 mi  Bukit China - 2.4 km / 1.5 mi  St. Peter s Church Melaka - 2.5 km / 1.5 mi  AEON Bandaraya Melaka Shopping Centre - 2.5 km / 1.6 mi  Kee Ann Food Street - 2.7 km / 1.7 mi  Poh San Teng Temple - 2.7 km / 1.7 mi  Sky Tower Melaka - 2.7 km / 1.7 mi  Malay Living Museum - 2.7 km / 1.7 mi  The Shore Oceanarium - 2.8 km / 1.7 mi  The Shore Shopping Gallery - 2.8 km / 1.8 mi  St. John s Fort - 3.2 km / 2 mi  Kampung Hulu Mosque - 3.3 km / 2.1 mi  Red Square - 3.3 km / 2.

In [29]:
print("There are", chroma_document_store._collection.count(), "hotels with mountain biking in Canada")
similar_documents = chroma_document_store.similarity_search(query="a cliffside trail",filter="and(eq(\"countryName\", \"UK\"), eq(\"HotelRating\", \"All\"))",k=10)
similar_documents

There are 1217 'hotels in the UK with trail running'


ValueError: Expected where to be a dict, got and(eq("countryName", "UK"), eq("HotelRating", "All"))

In [2]:
import os
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from langchain.tools import BaseTool, StructuredTool, tool
os.environ['OPENAI_API_KEY'] = ''
os.environ["OPENWEATHERMAP_API_KEY"] = ""


@tool
def search_weather(query: str) -> str:
    """A weather tool optimized for comprehensive up to date weather information. 
    Useful for when you need to answer questions about the weather, use this tool to answer questions about the weather for a specific location. 
    Look in context for the location to provide weather for"""
    weather = OpenWeatherMapAPIWrapper()
    weather_data = weather.run(query)
    return weather_data

search_weather.invoke('london')

'In london, the current weather is as follows:\nDetailed status: broken clouds\nWind speed: 4.12 m/s, direction: 230°\nHumidity: 80%\nTemperature: \n  - Current: 13.95°C\n  - High: 15.27°C\n  - Low: 12.53°C\n  - Feels like: 13.49°C\nRain: {}\nHeat index: None\nCloud cover: 75%'

In [ ]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

#wikipedia.run("HUNTER X HUNTER")

In [4]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
from langchain.chains.query_constructor.base import AttributeInfo
from langchain_openai import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_core.prompts import PromptTemplate, format_document, MessagesPlaceholder
from langchain.tools.retriever import create_retriever_tool
from langchain.tools import Tool
from langchain import hub
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent

os.environ["LANGCHAIN_API_KEY"] = ""
os.environ['OPENAI_API_KEY'] = ''
os.environ["OPENWEATHERMAP_API_KEY"] = ""
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_PROJECT'] = "ellie_chatbot_debug_jupyter" 
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small") 

chroma_document_store = Chroma( 
    collection_name="chapter8db_hotel_store", 
    embedding_function=embeddings_model, 
    persist_directory="./streamlit/chapter8db",
)
query = "four star hotels in uk, montacute who are happy to take dogs"
query = "four star hotels in Devon that don't accept hen parties"

query = "im looking for a bed and breakfast in aberdeen with restaurants nearby"
similar_documents = chroma_document_store.similarity_search(query=query,k=1)
print("YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY")
print(similar_documents)
print("There are", chroma_document_store._collection.count(), "in the collection")

metadata_field_info = [
    AttributeInfo(
        name="HotelRating",
        description="hotel rating One of ['FourStar', 'ThreeStar', 'TwoStar', 'All']",
        type="string",
    ),
    AttributeInfo(
        name="cityName",
        description="Name of the city",
        type="string",
    ),
    AttributeInfo(
        name="countryName",
        description="Name of the country",
        type="string"
    ),
]
document_content_description = "Reviews and Descriptions of hotels"
llm = OpenAI(temperature=0)


retriever = SelfQueryRetriever.from_llm(
        llm,
        chroma_document_store,
        document_content_description,
        metadata_field_info,
        verbose=True,
        search_type='similarity',
        search_kwargs={'k': 10})


template = """you can use the info below to answer questions. 
Helpful Answer: HotelName: {HotelName} CountryName: {countryName} CityName: {cityName} Address: {Address} HotelRating: {HotelRating} HotelWebsite: {HotelWebsiteUrl}\n\n{page_content}"""
  
prompt = PromptTemplate( 
    input_variables=["HotelName", "HotelRating","countryName","cityName","Address", "HotelWebsiteUrl"], 
    template=template, 
) 

hotel_search = create_retriever_tool(
    retriever,
    "provide_hotel_suggestions",
    "Searches and provides hotel suggestions by searching by question and any other metadata, return the Hotel name, Hotel Rating, Hotel City, Hotel Country, Description, Website and a short summary from the metadata.",
   document_prompt=prompt
)

docs = hotel_search.run(query)
print(query)
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
print(docs)



llm = ChatOpenAI(temperature=0.0, model='gpt-4-1106-preview',verbose=True)

tools = [hotel_search]

hub_prompt = hub.pull("hwchase17/openai-tools-agent")

memory = ConversationBufferWindowMemory(k=20, memory_key="chat_history", return_messages=True)

agent = create_openai_tools_agent(llm, tools, hub_prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
)

result = agent_executor.invoke({"input": query})
print(result["output"])

YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
[Document(page_content='have showers and complimentary toiletries.Dining : Satisfy your appetite for lunch or dinner at the bed & breakfast s restaurant, or stay in and take advantage of the room service (during limited hours). A complimentary continental breakfast is', metadata={'Address': 'Route Gorge Du Dades Ait Idir Boumalne Dades Ait Idir Boumalne Dades45145 Boumalne Dads', 'Attractions': 'Distances are displayed to the nearest 0.1 mile and kilometer.  Monkey Paw Canyon - 0.9 km / 0.5 mi  Monkey Fingers - 3.6 km / 2.3 mi  Dads Gorge - 9.1 km / 5.7 mi  At Oudinar Womens Weaving Cooperative - 16.3 km / 10.2 mi  Boumalne Mosque - 18.3 km / 11.4 mi  Dades Municipal Stadium - 18.9 km / 11.7 mi  Place des Festivites - 38.7 km / 24.1 mi  Abdellaoul Mosque - 38.8 km / 24.1 mi  Kasbah Ruins - 39.5 km / 24.5 mi  ', 'HotelCode': '1628701', 'HotelName': "Maison D'hotes Mogador Chez Habitants", 'HotelRating': 'TwoStar', 'HotelWebsiteUrl': '', 'Map': '31

In [ ]:
from langchain.agents import AgentType, initialize_agent
from langchain_community.utilities.openweathermap import OpenWeatherMapAPIWrapper
from langchain_openai import OpenAI, ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(temperature=0.0,openai_api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4-1106-preview")

agent = initialize_agent(
    tools=[search_weather,wikipedia], llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

agent.run("whats the weather like in London?")

In [30]:
from langchain.agents import AgentType, initialize_agent
from langchain_community.utilities.openweathermap import OpenWeatherMapAPIWrapper
from langchain_openai import OpenAI, ChatOpenAI, OpenAIEmbeddings


from langchain.chat_models import ChatOpenAI
from langchain.tools import tool
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools.retriever import create_retriever_tool
import os


chroma_document_store = Chroma( 
    collection_name="chapter8db_hotel_store", 
    embedding_function=embeddings_model, 
    persist_directory="chapter8db",
)





gpt4_llm = ChatOpenAI(temperature=0.0, model='gpt-4-1106-preview',
                              verbose=False)


retrive_tool = create_retriever_tool(
    chroma_document_store.as_retriever(),
    name="hotel_search",
    description="""Here I usually use the prompt tamplate for the RAG""",
    
)
tools = [wikipedia, retrive_tool]

template = """Provide hotel suggestions by searching for ones similar to the question based on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Provide hotel suggestions by searching for ones similar to the question based on the following context: {context}"),
        ("user", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

#prompt = ChatPromptTemplate.from_template(template)
# prompt = PromptTemplate(
#     template="Answer the user query.\\n{format_instructions}\\n",
#     input_variables=["query"])

agent = create_openai_tools_agent(gpt4_llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": "hotel in magog with mountain biking"})

C:\Users\AdrianThomspon\anaconda3\envs\python310\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


NameError: name 'wikipedia' is not defined

In [ ]:
metadata_field_info = [
    AttributeInfo(
        name="HotelRating",
        description="hotel rating One of ['FourStar', 'ThreeStar', 'TwoStar', 'All']",
        type="string",
    ),
    AttributeInfo(
        name="cityName",
        description="Name of the city",
        type="string",
    ),
    AttributeInfo(
        name="countryName",
        description="Name of the country",
        type="string"
    ),
]
document_content_description = "Reviews and Descriptions of hotels"
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    chroma_document_store,
    document_content_description,
    metadata_field_info,
    verbose=True,
    search_type='similarity',
    search_kwargs={'k': 10}
)


In [70]:

import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_core.prompts import PromptTemplate, format_document, ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory,ConversationBufferMemory
from langchain.tools import WikipediaQueryRun,BaseTool, StructuredTool, tool
from langchain_community.utilities import WikipediaAPIWrapper, OpenWeatherMapAPIWrapper
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import OpenAI

os.environ['OPENAI_API_KEY'] = ''
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ['LANGCHAIN_PROJECT'] = "book-chapter-8" 
os.environ["OPENWEATHERMAP_API_KEY"] = ""

embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small") 

llm = ChatOpenAI(temperature=0.0, model='gpt-4-1106-preview',verbose=True)


@tool
def search_weather(query: str) -> str:
    """A weather tool optimized for comprehensive up to date weather information. 
    Useful for when you need to answer questions about the weather, use this tool to answer questions about the weather for a specific location. 
    Look in context for the location to provide weather for"""
    weather = OpenWeatherMapAPIWrapper()
    weather_data = weather.run(query)
    return weather_data


chroma_document_store = Chroma( 
    collection_name="chapter8db_hotel_store", 
    embedding_function=embeddings_model, 
    persist_directory="chapter8db",
)

#retriever = chroma_document_store.as_retriever(search_kwargs={"k": 5})

template = """you can use the info below to answer questions.
Helpful Answer:"HotelName: {HotelName} HotelRating: {HotelRating} HotelWebsite: {HotelWebsiteUrl}\n\n{page_content}"""

prompt = PromptTemplate(
input_variables=["HotelName","HotelRating","HotelWebsiteUrl"],
template=template,
)


retriever_tool = create_retriever_tool(
    retriever,
    "search_hotel_suggestions",
    "Searches and provides hotel suggestions by searching by question, return the Hotel name, Hotel Rating, Website and a short summary from the metadata",
   document_prompt=prompt
)

# Create the Wikipedia API wrapper
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)

# Create the WikipediaQueryRun tool
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper,description="look up tourist information")

tools = [retriever_tool,search_weather,wikipedia]

hub_prompt = hub.pull("hwchase17/openai-tools-agent")

memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True)

agent = create_openai_tools_agent(llm, tools, hub_prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    return_intermediate_steps=False,
)

result = agent_executor.invoke(
    {
        "input": "can you suggest a fourstar hotel in the UK with some decent views and skiiing"
    }
)





> Entering new AgentExecutor chain...

Invoking: `search_hotel_suggestions` with `{'query': 'four-star hotel with views and skiing in the UK'}`


I found a four-star hotel in the UK that might suit your preferences:

**Hotel Name:** Macdonald Aviemore Hotel
**Hotel Rating:** 4 stars
**Website:** [Macdonald Aviemore Hotel](https://www.macdonaldhotels.co.uk/our-hotels/scotland/aviemore/macdonald-aviemore-hotel)
**Summary:** Located in the heart of the Cairngorms National Park, the Macdonald Aviemore Hotel offers stunning views and is close to the Aviemore Ski Resort. Guests can enjoy a range of outdoor activities including skiing during the winter months.

Please note that the availability of skiing is subject to seasonal conditions and it's always best to check the current weather and ski resort operations before planning your trip.

> Finished chain.


In [71]:
result = agent_executor.invoke(
    {
        "input": "do you have any more informatoin about that area"
    }
)

result['output']
memory





> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Cairngorms National Park`


Page: Cairngorms National Park
Summary: Cairngorms National Park (Scottish Gaelic: Pàirc Nàiseanta aThe Cairngorms National Park is a national park in the eastern Highlands of Scotland that was established in 2003. It covers an area of 4,528 square kilometers (1,748 sq mi), making it the largest national park in the British Isles. The park encompasses the Cairngorms mountain range and surrounding hills, with many of the UK's highest peaks found within its boundaries.

The Cairngorms are known for their stunning landscapes, diverse wildlife, and outdoor recreational opportunities. The park is a popular destination for hiking, mountain biking, and winter sports such as skiing and snowboarding. The Cairngorm Mountain ski area is one of Scotland's most renowned ski resorts and offers a variety of runs for different skill levels.

In addition to outdoor sports, the Cairngorms National Park is al

ConversationBufferWindowMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='can you suggest a fourstar hotel in the UK with some decent views and skiiing'), AIMessage(content="I found a four-star hotel in the UK that might suit your preferences:\n\n**Hotel Name:** Macdonald Aviemore Hotel\n**Hotel Rating:** 4 stars\n**Website:** [Macdonald Aviemore Hotel](https://www.macdonaldhotels.co.uk/our-hotels/scotland/aviemore/macdonald-aviemore-hotel)\n**Summary:** Located in the heart of the Cairngorms National Park, the Macdonald Aviemore Hotel offers stunning views and is close to the Aviemore Ski Resort. Guests can enjoy a range of outdoor activities including skiing during the winter months.\n\nPlease note that the availability of skiing is subject to seasonal conditions and it's always best to check the current weather and ski resort operations before planning your trip."), HumanMessage(content='do you have any more informatoin about that area'), AIMessage(content="The C

In [4]:

import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_core.prompts import PromptTemplate, format_document, ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory,ConversationBufferMemory
from langchain.tools import WikipediaQueryRun,BaseTool, StructuredTool, tool
from langchain_community.utilities import WikipediaAPIWrapper, OpenWeatherMapAPIWrapper
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import OpenAI
import json
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI

os.environ['OPENAI_API_KEY'] = ''
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ['LANGCHAIN_PROJECT'] = "book-chapter-8" 
os.environ["OPENWEATHERMAP_API_KEY"] = ""


@tool
def search_weather(query: str) -> str:
    """A weather tool optimized for comprehensive up to date weather information. 
    Useful for when you need to answer questions about the weather, use this tool to answer questions about the weather for a specific location. 
    Look in context for the location to provide weather for"""
    weather = OpenWeatherMapAPIWrapper()
    weather_data = weather.run(query)
    return weather_data

# Define the LLM prompt
instructions = """You are an assistant. Always start by asking the user for their holiday preferences. If the user provides preferences, store them in a variable called `preferences`. If the user does not provide preferences, ask them again. Once you have the user's preferences, generate a response based on those preferences."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

# Define the LLM model
llm = ChatOpenAI(temperature=0)

# Define the tools
tools = [search_weather]

# Create the agent
agent = create_openai_functions_agent(llm, tools, prompt)

# Define the output parser
def output_parser(output):
    # Parse the output from the LLM
    output_json = json.loads(output)
    preferences = output_json["preferences"]

    # Generate a response based on the user's preferences
    response = "I have received your holiday preferences. I will use this information to generate responses that are tailored to your interests."

    # Return the response
    return response

# Create the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

# Invoke the agent executor
agent_executor.invoke({"input": "What are your holiday preferences?"})




> Entering new AgentExecutor chain...
What are your holiday preferences?

> Finished chain.


{'input': 'What are your holiday preferences?',
 'output': 'What are your holiday preferences?'}

In [5]:
agent_executor.invoke({"input": "I like mountain biking"})



> Entering new AgentExecutor chain...
That's great to hear! Do you have any specific preferences for your mountain biking holiday, such as a preferred location or type of terrain?

> Finished chain.


{'input': 'I like mountain biking',
 'output': "That's great to hear! Do you have any specific preferences for your mountain biking holiday, such as a preferred location or type of terrain?"}

In [7]:
hub_prompt = hub.pull("hwchase17/openai-tools-agent")
hub_prompt



ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlacehold